In [ ]:
vlib = c("tidyverse", "ggpubr", "patchwork","tidyseurat","ggrastr", "ggsci")
lapply(vlib, require, character.only = TRUE, quietly = TRUE) |> suppressMessages()

if (length(list.files("/data/podo")) == 0) {
  base_dir = "/data/nfs_dell/project_HS/"
} else {
  base_dir = "/data/podo/Projects/project_HS/"
}

chr_wd = paste0(base_dir, "202310-sceqtl_v6/")
setwd(chr_wd)

saveRDS.gz <- function(object, file, threads=4) {
  con <- pipe(paste0("pigz -p", threads, " > ", file), "wb")
  saveRDS(object, file = con)
  close(con)
}
readRDS.gz <- function(file, threads = parallel::detectCores()) {
  con <- pipe(paste0("pigz -d -c -p", threads, " ", file))
  object <- readRDS(file = con)
  close(con)
  return(object)
}

## SF6C

In [ ]:
df_ME.aida = read_delim("scRNAseq_downstream_scenic/projected_aida.module_kME.txt.gz", delim = "\t") %>%
  filter(cellID %in% df_meta_cd8_aida$cellID) %>%
  mutate(data = "AIDA_CTRL")
df_ME.ref = GetMEs(seurat_obj.ref) %>%
  as.data.frame() %>%
  rownames_to_column("cellID") %>%
  filter(cellID %in% df_meta_cd8_ref$cellID) %>%
  mutate(data = "LungCancer")

In [ ]:
seurat_obj_GSE169246@meta.data %>% distinct(tissue, response, time, regimen)
df_meta_cd8_GSE169246  = seurat_obj_GSE169246@meta.data %>%
  rownames_to_column("cellID") %>%
  filter(predicted.celltype.l1 == "CD8 T", tissue == "blood", time == "Pre-treatment")

df_ME.GSE169246 = GetMEs(seurat_obj_GSE169246) %>%
  as.data.frame() %>%
  rownames_to_column("cellID") %>%
  filter(cellID %in% df_meta_cd8_GSE169246$cellID) %>%
  mutate(data = "TNBC")

In [ ]:
seurat_obj_GSE200996 = readRDS.gz("scRNA_wgcna_replication/GSE200996/seurat_obj.merged.RDS")
df_meta_add.GSE200996 = read_delim("scRNA_wgcna_replication/GSE200996/supplementary_metadata.txt")

In [ ]:
df_meta_cd8_GSE200996  = seurat_obj_GSE200996@meta.data %>%
  rownames_to_column("cellID") %>%
  left_join(., df_meta_add.GSE200996) %>%
  filter(predicted.celltype.l1 == "CD8 T", time == "baseline")

df_ME.GSE200996 = GetMEs(seurat_obj_GSE200996) %>%
  as.data.frame() %>%
  rownames_to_column("cellID") %>%
  filter(cellID %in% df_meta_cd8_GSE200996$cellID) %>%
  mutate(data = "HNSCC")

In [ ]:
p1 = rbind(df_ME.aida, df_ME.ref) %>%
  rbind(., df_ME.GSE169246) %>%
  rbind(., df_ME.GSE200996) %>%
  mutate(
    data = factor(data, levels = c("AIDA_CTRL", "TNBC", "HNSCC", "LungCancer")),
    projected = ifelse(data == "LungCancer", "our results", "projected")
  ) %>%
  ggplot(aes(x = data, y = brown)) +
  geom_violin(width = 0.85) +
  geom_boxplot(width = 0.30, alpha = 0.5, outlier.shape = NA) +
  xlab("Dataset") +
  ylab("Brown module expression in CD8 ") +
  facet_grid(~projected, scales = "free_x", space = "free") +
  theme_pubr() +
  theme(
    axis.title = element_text(size = 15),
    axis.text = element_text(size = 13),
  )
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)
p1

p1 %>% ggsave(
  "figure_prep/supplementary/sf6C_compareAIDA_CD8.pdf",
  ., height = 4, width = 5, dpi = 300
)

## SF6B

In [ ]:
df_pairwise_test = left_join(df_ME.GSE200996, df_meta_cd8_GSE200996) 
pairwise.wilcox.test(df_pairwise_test$brown, df_pairwise_test$Patho_Bins, p.adjust.method = "bonferroni", correct = FALSE)

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4, repr.plot.res = 300)
p_sf6_hnscc = left_join(df_ME.GSE200996, df_meta_cd8_GSE200996) %>%
mutate(Patho_Bins = factor(Patho_Bins, levels = c( "High","Medium","Low"))) %>%
filter(is.na(Patho_Bins) == FALSE) %>%
ggplot(aes(x = Patho_Bins, y = brown)) +
  geom_violin(width = 0.85) +
  geom_boxplot(width = 0.30, alpha = 0.5, outlier.shape = NA) +
  stat_compare_means() +
  xlab("pathological response") +
  ylab("GSE200996, CD8 Brown Module") +
  theme_pubr()

p_sf6_hnscc %>% ggsave(
  "figure_prep/supplementary/sf6_wgcna_projection_hgcna.pdf",
  ., height = 4, width = 4, dpi = 300
)

In [ ]:
p2 = read_delim("assets/cell_proportion.projected_sets.txt.gz", delim = "\t") %>%
  mutate(CD8_subtype = str_replace(CD8_subtype, "_", " ")) %>%
  filter(!CD8_subtype == "CD8 Proliferating") %>%
  ggplot(aes(x = data, y = n, fill = CD8_subtype)) +
    geom_bar(stat = "identity", position = "fill") +
    theme_pubr(x.text.angle = 45, legend = "right") +
    scale_fill_simpsons()
options(repr.plot.width = 4, repr.plot.height = 4, repr.plot.res = 300)
p2 %>% ggsave(
  "figure_prep/supplementary/sf6_tem_proportion.pdf",
  ., height = 4, width = 4, dpi = 300
)